In [1]:
from ConectionSQL import SQL
import pandas as pd
from Tempo import DataHora

sql=SQL('Netfeira','sqlserver','MOINHO','192.168.0.252')

conectando=sql.ConexaoSQL()

data=DataHora()

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
data_atual=data.HoraAtual()

ano=data_atual.year

mes=data_atual.month

dia=data_atual.day

In [4]:
querys={
    
'Estático':"""

SELECT * FROM netfeira.vw_estatico
WHERE YEAR([Data de Faturamento])=YEAR(GETDATE()) AND MONTH([Data de Faturamento])=MONTH(GETDATE())
AND [Tipo de Operação]='VENDAS'


""",
    
'Meta':"""


SET NOCOUNT ON;

Declare @Coluna_Pivot AS NVARCHAR(MAX),@Comando AS NVARCHAR(MAX)

SET @Coluna_Pivot=STUFF((SELECT DISTINCT ', ' + cd_tp_prev FROM prev_vda FOR XML PATH('')),1,1,'')

SET @Comando='
SELECT * FROM (
SELECT prev_vda.cd_prev_vda,mes_ref,CONVERT(VARCHAR(7),mes_ref,120) mes_meta,
cd_tp_prev,it_prev_vda.cd_vend,
valor
FROM prev_vda
LEFT JOIN it_prev_vda ON prev_vda.cd_prev_vda=it_prev_vda.cd_prev_vda AND prev_vda.cd_tp_prev=it_prev_vda.cd_tp_prev_det)linha
PIVOT(SUM(valor) FOR cd_tp_prev IN ('+@Coluna_Pivot+'))coluna'

EXECUTE(@Comando)


""",
    
'Equipe':"""

SELECT * FROM netfeira.vw_equipe


""",
    
'Vendedor':"""


SELECT * FROM netfeira.vw_vendedor

""",
    
'Calendario':"""

DECLARE @DTInicial AS DATETIME,@DTFinal AS DATETIME

SET @DTInicial='2018-01-01'
SET @DTFinal= CONCAT(YEAR(GETDATE())+1,'-01-','01')

;WITH Calendario (Datas) AS(

SELECT @DTInicial
UNION ALL
SELECT Datas+1
FROM Calendario WHERE  Datas+1<@DTFinal
)

SELECT CONVERT(DATETIME,CAST(Datas AS DATE),101) AS 'Data',YEAR(Datas) AS 'Ano',MONTH(Datas) AS 'Cód. Mês',
CHOOSE(MONTH(Datas),'JANEIRO','FEVEREIRO','MARÇO','ABRIL','MAIO','JUNHO','JULHO','AGOSTO','SETEMBRO','OUTUBRO','NOVEMBRO','DEZEMBRO') AS 'Mês',
CHOOSE(MONTH(Datas),'JAN','FEV','MAR','ABR','MAI','JUN','JUL','AGO','SET','OUT','NOV','DEZ') AS 'Mês Resumido',
DAY(Datas) AS 'Dia',CONVERT(VARCHAR(7),Datas,120) AS 'Mês Meta',
DATEPART(DW,Datas) AS 'Cód. Semana',CHOOSE(DATEPART(DW,Datas),'DOM','SEG','TER','QUAR','QUI','SEX','SÁB') AS 'Semana',
CASE WHEN DATEPART(DW,Datas) IN (7,1) THEN 0 ELSE 1 END AS 'Dias Úteis',
CASE WHEN MONTH(Datas)<=3 THEN '1º TRIM' WHEN MONTH(Datas)<=6 THEN '2º TRIM' WHEN MONTH(Datas)<=9 THEN '3º TRIM' WHEN MONTH(Datas)<=12 THEN '4º TRIM' END AS 'Trimestre Ano',
CASE WHEN MONTH(Datas)<=6 THEN '1º SEM' ELSE '2º SEM' END AS 'Semestre Ano',DATEPART(WEEK,Datas) AS 'Semana Ano',
CONVERT(VARCHAR,DAY(Datas))+'/'+CONVERT(VARCHAR,MONTH(Datas)) AS 'Mês Base'
FROM Calendario OPTION(MAXRECURSION 10000)


""",
    
'Feriado':"""

DECLARE @DateInicial AS DATETIME,@DateFinal AS DATETIME

SET @DateInicial='2018-01-01'
SET @DateFinal=CONCAT(YEAR(GETDATE())+1,'-01-01')

;WITH TabDatas(dt_calend) AS (
SELECT @DateInicial
UNION ALL
SELECT dt_calend+1 FROM TabDatas WHERE dt_calend+1<@DateFinal),

TabCalendario (dt_calend,mes_base) AS(

SELECT CONVERT(DATETIME,CAST(dt_calend AS DATE),101),CONVERT(VARCHAR,DAY(dt_calend))+'/'+CONVERT(VARCHAR,MONTH(dt_calend)) AS mes_base
FROM TabDatas 
),

TabFeriado (dt_calend,mes_base,dias_feriado) AS (

SELECT dt_calend,mes_base,
CASE WHEN mes_base IN ('1/1','10/4','21/4','1/5','11/6','7/9','12/10','2/11','15/11','25/12','8/4','16/6') THEN 0 ELSE 1 END 
FROM TabCalendario)

SELECT DISTINCT mes_base AS 'Mês Base'
FROM TabFeriado
WHERE dias_feriado=0
ORDER BY 1
OPTION(MAXRECURSION 10000)

""",
    
'Produto':"""

SELECT * FROM netfeira.vw_produto


""",
    
'Aberto':"""


SELECT * FROM netfeira.vw_aberto


""",
    
'TargetEstatico':
    
"""

SELECT * FROM netfeira.vw_targetestatico
WHERE YEAR([Data de Faturamento])=YEAR(GETDATE()) AND MONTH([Data de Faturamento])=MONTH(GETDATE())

"""
    
    
}

In [5]:
tabelas_df=dict()

for tabela,query in querys.items():
    
    tabelas_df[tabela]=pd.read_sql(query,conectando)    
    
    pass

In [6]:
#dias úteis

tabelas_df['Calendario']['Feriado']=tabelas_df['Calendario'].apply(lambda info: len(tabelas_df['Feriado']['Mês Base'].loc[tabelas_df['Feriado']['Mês Base']==info['Mês Base']].unique().tolist()),axis=1)

tabelas_df['Calendario']['Dias Úteis']=tabelas_df['Calendario'].apply(lambda info: 0 if info['Feriado']==1 else info['Dias Úteis'],axis=1)

tabelas_df['Calendario'].drop(columns=['Feriado'],inplace=True)

dias_uteis=tabelas_df['Calendario'][['Dias Úteis']].loc[(tabelas_df['Calendario']['Ano']==ano)&(tabelas_df['Calendario']['Cód. Mês']==mes)].sum()

dias_trabalhados=tabelas_df['Calendario'][['Dias Úteis']].loc[(tabelas_df['Calendario']['Ano']==ano)&(tabelas_df['Calendario']['Cód. Mês']==mes)&(tabelas_df['Calendario']['Dia']<=dia)].sum()

dias_restante=dias_uteis[0]-dias_trabalhados[0]

dias_trabalhados=dias_trabalhados[0]

dias_uteis=dias_uteis[0]

In [7]:
vendas_df=tabelas_df['Estático']

In [8]:
df_aberto=tabelas_df['Aberto']

In [9]:
metas_df=tabelas_df['Meta'][['cd_vend','FATUVL']].loc[(tabelas_df['Meta']['mes_ref'].dt.year==ano)&(tabelas_df['Meta']['mes_ref'].dt.month==mes)].groupby(['cd_vend','FATUVL'],as_index=False).sum()

metas_df.rename(columns={'cd_vend':'ID Vendedor','FATUVL':'Meta R$'},inplace=True)

metas_df=metas_df.merge(tabelas_df['Vendedor'],on='ID Vendedor',how='left')[['ID Vendedor','Vendedor','ID Equipe','Meta R$']]

metas_df=metas_df.merge(tabelas_df['Equipe'],on='ID Equipe',how='inner')[['ID Vendedor','Vendedor','Equipe','Supervisor','Meta R$']]

metas_df['Meta Diária']=metas_df.apply(lambda info: round(info['Meta R$']/dias_uteis,2),axis=1)

metas_df['Faturado R$']=metas_df.apply(lambda info: round(vendas_df['Total Geral'].loc[(vendas_df['Data de Faturamento'].dt.month==mes)&(vendas_df['ID Vendedor']==info['ID Vendedor'])&(~vendas_df['Status do Pedido'].isin(['EM ABERTO']))].sum(),2),axis=1)

metas_df['Realizado R$']=metas_df.apply(lambda info: df_aberto['Total Geral'].loc[(df_aberto['ID Vendedor']==info['ID Vendedor'])].sum(),axis=1)

metas_df['Dif Meta']=metas_df.apply(lambda info: info['Faturado R$']-info['Meta R$'],axis=1)

metas_df['Dif Realizado']=metas_df.apply(lambda info: info['Realizado R$']-info['Meta Diária'],axis=1)

supervisor_df=metas_df[['Supervisor','Meta Diária','Realizado R$']].groupby(['Supervisor'],as_index=False).sum()

supervisor_df['Dif Realizado']=supervisor_df.apply(lambda info: info['Meta Diária']-info['Realizado R$'],axis=1)

supervisor_df['Realizado %']=supervisor_df.apply(lambda info: round(info['Realizado R$']/info['Meta Diária'],4)*100,axis=1)

supervisor_df
    

,Supervisor,Meta Diária,Realizado R$,Dif Realizado,Realizado %
0,CARLOS PASSOS,17642.86,20296.29,-2653.43,115.04
1,DOUGLAS TEDESCHI,25295.24,36382.62,-11087.38,143.83
2,FERNANDA MEIRA,24900.00,33210.85,-8310.85,133.38
3,ROGERIO FELIPIM,61492.87,99671.85,-38178.98,162.09
4,ULISSES BACCI,39328.59,51242.07,-11913.48,130.29


In [10]:
metas_df[['Equipe','Meta Diária','Realizado R$','Dif Realizado']].groupby(['Equipe'],as_index=False).sum()

,Equipe,Meta Diária,Realizado R$,Dif Realizado
0,E-COMMERCE,959.53,1132.45,172.92
1,EQUIPE 1 (REP),25757.15,31169.32,5412.17
2,EQUIPE 11 (LS),1519.05,0.00,-1519.05
3,EQUIPE 2 (REP),16276.20,21450.28,5174.08
4,EQUIPE 3 (CLT),23052.39,29791.79,6739.40
5,EQUIPE 4 (VAR),25295.24,36382.62,11087.38
6,EQUIPE 5 (INT),25538.09,60895.49,35357.40
7,EQUIPE 6 (CLT),17642.86,20296.29,2653.43
8,EQUIPE 7 (LOJA),3742.86,5726.66,1983.80
9,EQUIPE 8 (ATEND),3976.19,747.93,-3228.26


In [11]:
df_aberto['Total Geral'].sum()

241287.68

In [12]:
df_aberto[['ID Vendedor','Total Geral']].groupby(['ID Vendedor'],as_index=False).sum()

,ID Vendedor,Total Geral
0,ADANTAS,46113.28
1,ADELIALO,7288.74
2,ADEMIRFE,5429.12
3,ALERUIZ,2118.52
4,AVULSO,747.93
5,BRUNA,7003.95
6,CARLOSH,332.38
7,CELSOCAR,8762.61
8,CLAUDIOP,5211.73
9,CRIS,3286.96


In [13]:
tabelas_df['Estático'].merge(tabelas_df['Vendedor'],on='ID Vendedor',how='inner')[['Numero do Pedido', 'NFe', 'ID Cliente', 'ID Vendedor','ID Equipe',
       'Tipo de Pedido', 'Tipo de Operação', 'Tabela de Preço',
       'Data de Emissão', 'Data de Faturamento', 'Status do Pedido', 'ID NFe',
       'SKU', 'Seq', 'Qtde', 'Unid. VDA', 'Qtde VDA', 'Valor VDA',
       'Total Geral', 'Custo da Útima Entrada', 'Peso Bruto', 'Peso Líquido',
       'Preço da Tabela', '% Desconto', '% Comissão', 'Comissão R$']]

,Numero do Pedido,NFe,ID Cliente,ID Vendedor,ID Equipe,Tipo de Pedido,Tipo de Operação,Tabela de Preço,Data de Emissão,Data de Faturamento,Status do Pedido,ID NFe,SKU,Seq,Qtde,Unid. VDA,Qtde VDA,Valor VDA,Total Geral,Custo da Útima Entrada,Peso Bruto,Peso Líquido,Preço da Tabela,% Desconto,% Comissão,Comissão R$
0,410402,944174,1671,RODRIGOA,0004,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,2022-07-05,2022-07-06,FATURADO,455126,1106,1,18.0,UN,18.0,17.77,319.86,195.84,23.40,23.40,17.77,0.0000,0.01,3.20
1,410402,944174,1671,RODRIGOA,0004,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,2022-07-05,2022-07-06,FATURADO,455126,955,2,8.0,UN,8.0,5.72,45.76,28.00,1.72,1.64,5.72,0.0000,0.01,0.46
2,410402,944174,1671,RODRIGOA,0004,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,2022-07-05,2022-07-06,FATURADO,455126,956,3,8.0,UN,8.0,5.72,45.76,28.00,1.72,1.64,5.72,0.0000,0.01,0.46
3,410402,944174,1671,RODRIGOA,0004,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,2022-07-05,2022-07-06,FATURADO,455126,954,4,8.0,UN,8.0,5.72,45.76,28.00,1.72,1.64,5.72,0.0000,0.01,0.46
4,410402,944174,1671,RODRIGOA,0004,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,2022-07-05,2022-07-06,FATURADO,455126,959,5,6.0,UN,6.0,19.60,117.60,72.00,5.76,5.46,19.60,0.0000,0.01,1.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5693,409836,943491,28120,REGIS,0002,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,2022-07-01,2022-07-04,FATURADO,454388,1003,2,18.0,CX,2.0,157.77,315.54,194.41,22.68,21.60,17.90,0.0207,0.05,15.78
5694,409836,943491,28120,REGIS,0002,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,2022-07-01,2022-07-04,FATURADO,454388,730,3,20.0,CX,2.0,165.30,330.60,235.21,25.20,24.00,17.06,0.0311,0.05,16.53
5695,409836,943491,28120,REGIS,0002,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,2022-07-01,2022-07-04,FATURADO,454388,486,4,30.0,CX,3.0,127.30,381.90,249.04,31.50,30.00,13.07,0.0260,0.05,19.10
5696,411234,944617,28120,REGIS,0002,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,2022-07-07,2022-07-08,FATURADO,455633,469,1,25.0,CX,5.0,74.50,372.50,243.01,31.50,30.00,15.30,0.0261,0.05,18.63


In [14]:
tabelas_df['Aberto'].columns

Index(['Data do Pedido', 'Pedido', 'ID Cliente', 'ID Vendedor', 'SKU', 'Qtde',
       'Unid. VDA', 'Qtde. VDA', 'Valor Unitário', 'Total Geral', 'Peso Bruto',
       'Peso Líquido', 'Situação', 'Custo CUE'],
      dtype='object')

In [15]:
tabelas_df['Aberto']=tabelas_df['Aberto'].merge(tabelas_df['Vendedor'],on='ID Vendedor',how='inner')[['Data do Pedido', 'Pedido', 'ID Cliente', 'ID Vendedor','ID Equipe', 'SKU', 'Qtde',
       'Unid. VDA', 'Qtde. VDA', 'Valor Unitário', 'Total Geral', 'Peso Bruto',
       'Peso Líquido', 'Situação']]

In [16]:
tabelas_df['Equipe']

,ID Equipe,Equipe,Status,Supervisor
0,N/1,GERÊNCIA,ATIVO,ROGERIO FELIPIM
1,0001,EQUIPE 1 (REP),ATIVO,ROGERIO FELIPIM
2,0002,EQUIPE 2 (REP),ATIVO,ULISSES BACCI
3,0003,EQUIPE 3 (CLT),ATIVO,ULISSES BACCI
4,0004,EQUIPE 4 (VAR),ATIVO,DOUGLAS TEDESCHI
5,0005,EQUIPE 5 (INT),ATIVO,ROGERIO FELIPIM
6,0006,EQUIPE 6 (CLT),ATIVO,CARLOS PASSOS
7,0007,EQUIPE 7 (LOJA),ATIVO,ROGERIO FELIPIM
8,0008,EQUIPE 8 (ATEND),ATIVO,ROGERIO FELIPIM
9,0009,EQUIPE 9 (ATIVO),ATIVO,FERNANDA MEIRA


In [17]:
tabelas_df['Aberto'].loc[tabelas_df['Aberto']['ID Equipe']=='0009']

,Data do Pedido,Pedido,ID Cliente,ID Vendedor,ID Equipe,SKU,Qtde,Unid. VDA,Qtde. VDA,Valor Unitário,Total Geral,Peso Bruto,Peso Líquido,Situação
19,2022-07-11,411562,9189,RAQUELSO,0009,440,60.0,UN,60.0,24.58,1474.80,75.60,72.00,AB
20,2022-07-11,411562,9189,RAQUELSO,0009,486,40.0,UN,40.0,10.95,438.00,42.00,40.00,AB
21,2022-07-11,411583,26746,RAQUELSO,0009,415,10.0,UN,10.0,25.01,250.10,10.20,10.20,AB
22,2022-07-11,411583,26746,RAQUELSO,0009,440,10.0,UN,10.0,27.00,270.00,12.60,12.00,AB
23,2022-07-11,411583,26746,RAQUELSO,0009,1079,10.0,UN,10.0,50.00,500.00,12.60,12.00,AB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,2022-07-11,411761,26602,MAYRAQS,0009,1115,32.0,UN,32.0,13.09,418.88,19.20,19.20,AB
476,2022-07-11,411761,26602,MAYRAQS,0009,9550,22.0,CX,1.0,222.20,222.20,8.80,8.80,AB
477,2022-07-11,411761,26602,MAYRAQS,0009,1113,72.0,CX,6.0,172.92,1037.52,51.84,51.84,AB
478,2022-07-11,411761,26602,MAYRAQS,0009,9549,44.0,CX,2.0,194.96,389.92,17.60,17.60,AB


In [18]:
tabelas_df['TargetEstatico']=tabelas_df['TargetEstatico'].loc[tabelas_df['TargetEstatico']['Tipo de Operação']!='OUTROS']

tabelas_df['TargetEstatico'].columns

Index(['Data de Emissão', 'Data de Faturamento', 'Pedido', 'Nfe', 'ID Empresa',
       'ID Cliente', 'ID Vendedor', 'Tipo de Pedido', 'Tipo de Operação',
       'SKU', 'Qtde', 'Unid. VDA', 'Qtde VDA', 'Valor VDA', 'Total Geral',
       'Total Venda', 'Comissão R$', 'Margem Bruta R$', 'Cad Vendedor',
       'Situação', 'Peso Bruto KG', 'Peso Líquido KG'],
      dtype='object')

In [19]:
tabelas_df['TargetEstatico']['Tipo de Operação']=tabelas_df['TargetEstatico']['Tipo de Operação'].str.strip()

faturado=round(tabelas_df['TargetEstatico']['Total Venda'].loc[tabelas_df['TargetEstatico']['Situação']!='EM ABERTO'].sum(),2)

margem_bruta=round(tabelas_df['TargetEstatico']['Margem Bruta R$'].loc[(tabelas_df['TargetEstatico']['Situação']!='EM ABERTO')&(~tabelas_df['TargetEstatico']['Tipo de Operação'].isin(['BONIFICAÇÃO','AMOSTRA']))].sum(),2)

perc_bruto=round(margem_bruta/faturado,4)

perc_bruto

0.3049

In [25]:
tabelas_df['TargetEstatico']=tabelas_df['TargetEstatico'].merge(tabelas_df['Produto'],on='SKU',how='inner')[['Data de Emissão', 'Data de Faturamento', 'Pedido', 'Nfe', 'ID Empresa',
       'ID Cliente', 'ID Vendedor', 'Tipo de Pedido', 'Tipo de Operação',
       'SKU','Produto','Fabricante','Departamento','Seção','Categoria','Linha', 'Qtde', 'Unid. VDA', 'Qtde VDA', 'Valor VDA', 'Total Geral',
       'Total Venda', 'Comissão R$', 'Margem Bruta R$', 'Cad Vendedor',
       'Situação', 'Peso Bruto KG', 'Peso Líquido KG']]

In [37]:
fabric_df=tabelas_df['TargetEstatico'][['Fabricante','Total Venda','Margem Bruta R$']].loc[tabelas_df['TargetEstatico']['Situação']!='EM ABERTO'].groupby(['Fabricante'],as_index=False).sum()

fabric_df.sort_values('Total Venda',ascending=False,ignore_index=True,inplace=True)

fabric_df

,Fabricante,Total Venda,Margem Bruta R$
0,DE MARCHI,463562.20,149489.35
1,MCCAIN,368048.22,92770.99
2,ARYZTA,82272.61,21296.81
3,FRAGOLE,31907.43,11626.83
4,MATHEUS,19743.16,8305.16
5,DAUCY,13746.12,4413.69
6,REGILU,10565.22,3689.79
7,BUONO GEL,10034.36,3750.33
8,GRANO,8277.20,2838.39
9,PRACTICE,6716.55,2457.22
